In [45]:
import numpy as np
import os
# os.getcwd()
# os.chdir("/home/minion/Desktop/ML/HW1/CODE/")

# Distances seem accurate now, but the knn_predictions function is broken,
# because it only returns a total of 3 predictions, when there should be a total of 1000.
#

def knn_distances(xTrain, xTest, k):  # EUCLIDEAN

    distances = []
    indices = []
#     print(xTrain.shape, xTest.shape)
    for row_in_test in xTest: 
        new_dist = np.linalg.norm(xTrain - row_in_test, axis=1)
        new_index = np.argpartition(new_dist, min(k,xTrain.shape[0]-1) )[0:min(k,xTrain.shape[0]-1)]
        indices.append(new_index)
        distances.append(new_dist[new_index])
    # Return the top 3 closest distance for each row in the dev/test set.
    return np.array(indices), np.array(distances)


def knn_predictions(xTrain, yTrain, xTest, k):
#     xTrain: (5000,92)  - xTest (1000,92)  -> 1000,k
    indices, distances = knn_distances(xTrain, xTest, k)
    yTrain = yTrain.flatten()
    rows, columns = indices.shape
    predictions = list()
    for i in range(rows):
        temp=list(yTrain[indices[i,:]])
        predictions.append(max(temp, key=temp.count))
    predictions = np.array(predictions)
    return predictions


def calculate_error_rate(yTest, predictions):
    misclassified = 0

    for i, y in enumerate(yTest):  # For each observation and its index in the dev set
        h = predictions[i] * y  # Create a flag to check if the prediction is right.

        if h <= 0:  # If prediction is below or equal to zero, then it is wrong.
            misclassified += 1  # Add 1 to the misclassification counter.
#     print(yTest.shape[0])
    return misclassified/(yTest.shape[0])


def load_binarized_features(filename, num_rows, num_features):

    lines = open(filename).readlines()
    lines = [line.strip().split(", ") for line in lines]

    loaded_data = [[value for idx, value in enumerate(line) if idx not in [0, 7, 9]] for line in lines]
    # index 0, 7 are not categorical fields, 9 is target

    mapping = {}
    new_data = []

    for row in loaded_data:
        new_row = []
        for j, x in enumerate(row):
            feature = (j, x)
            if feature not in mapping:
                mapping[feature] = len(mapping)  # insert a new feature into the index
            new_row.append(mapping[feature])
        new_data.append(new_row)

    binary_data = np.zeros((num_rows, num_features + 2))
    # store normalized numerical values
    for idx, row in enumerate(new_data):
        for jdx in row:
            binary_data[idx][jdx] = 1
        binary_data[idx][num_features] = int(lines[idx][0]) / 50
        binary_data[idx][num_features + 1] = int(lines[idx][7]) / 50

    return binary_data


def load_labels(filename):
    lines = open(filename).readlines()
    lines = [line.strip().split(", ") for line in lines]

    labels = [[value for idx, value in enumerate(line) if idx in [9]] for line in lines]
    labels = [val for sublist in labels for val in sublist]
    y_array = []

    for label in labels:
        if label == '<=50K':
            y_array.append(-1)
        elif label == '>50K':
            y_array.append(1)

    y_array = np.array(y_array)
    return y_array



In [47]:
import time

    
def main():

    filenames = ['income.train.txt.5k', 'income.dev.txt']
    with open('combined.txt', 'w') as outfile:
        for f in filenames:
            with open(f) as infile:
                for line in infile:
                    outfile.write(line)

    filename = 'combined.txt'
    num_rows = 6000
    num_features = 90

    binarized_features = load_binarized_features(filename, num_rows, num_features)

    xTrain = binarized_features[0:5000]
    xDev = binarized_features[5000:6001]

    labels = load_labels(filename)

    yTrain = labels[0:5000] ## Should this be converted to numpy?
    yDev = labels[5000:6001]  ## Should this be converted to numpy?

    # Run K-NN


    for k in [1,3,5,7,9,99,999,9999]:
        start_time = time.time()
        indices, distances = knn_distances(xTrain, xDev, k)
        predictions = list(knn_predictions(xTrain, yTrain, xDev, k))
        positive_rate= round(predictions.count(1)/len(predictions),2)
        dev_err=round(calculate_error_rate(yDev, predictions)*100,2)
        print(f'k={k} dev_err {dev_err}% (+:{positive_rate}%)')
        print("--- %s seconds ---" % (time.time() - start_time))
        
if __name__ == "__main__":
    main()

k=1 dev_err 23.3% (+:0.27%)
--- 4.607374906539917 seconds ---
k=3 dev_err 19.2% (+:0.26%)
--- 4.314774036407471 seconds ---
k=5 dev_err 17.7% (+:0.25%)
--- 4.379318952560425 seconds ---
k=7 dev_err 16.5% (+:0.24%)
--- 4.308051109313965 seconds ---
k=9 dev_err 16.0% (+:0.23%)
--- 4.305510997772217 seconds ---
k=99 dev_err 15.8% (+:0.19%)
--- 5.069097995758057 seconds ---
k=999 dev_err 17.9% (+:0.11%)
--- 38.659247159957886 seconds ---


ValueError: kth(=5000) out of bounds (5000)

In [44]:
import platform
print(platform.python_version())

3.7.6
